In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("tiki_product.csv", dtype={"id": str})

In [4]:
del df["Unnamed: 0"]

In [5]:
df = df.drop_duplicates()

In [6]:
df["date_created"] = df["date_created"].clip(upper=8*365)

In [7]:
df["loaded_date"] = pd.to_datetime("2024-07-18 00:00:00")

In [8]:
product_sources = {
    "vietnamese_tiki_products_fashion_accessories.csv": "Phụ kiện thời trang",
    "vietnamese_tiki_products_women_shoes.csv": "Giày nữ",
    "vietnamese_tiki_products_men_shoes.csv": "Giày nam",
    "vietnamese_tiki_products_backpacks_suitcases.csv": "Balo & vali",
    "vietnamese_tiki_products_women_bags.csv": "Túi xách nữ",
    "vietnamese_tiki_products_men_bags.csv": "Túi xách nam"
}
df["product_source"] = df.source.map(product_sources)

In [9]:
df.columns

Index(['id', 'name', 'description', 'original_price', 'price',
       'fulfillment_type', 'brand', 'review_count', 'rating_average',
       'favourite_count', 'pay_later', 'current_seller', 'date_created',
       'number_of_images', 'vnd_cashback', 'has_video', 'category',
       'quantity_sold', 'source', 'loaded_date', 'product_source'],
      dtype='object')

## Thông tin các trường dữ liệu sản phẩm

- **id**: Mã định danh sản phẩm duy nhất  
- **name**: Tên sản phẩm  
- **description**: Mô tả sản phẩm  
- **original_price**: Giá gốc (chưa áp dụng giảm giá)  
- **price**: Giá hiện tại (có thể đã áp dụng hoặc chưa áp dụng giảm giá)  
- **fulfillment_type**: Hình thức giao hàng — `dropship` / `seller_delivery` / `tiki_delivery`  
  - **dropship**: Hàng được chuyển từ (người bán → kho TIKI → khách hàng), đóng gói bởi TIKI  
  - **seller_delivery**: Hàng được chuyển trực tiếp từ (người bán → khách hàng), đóng gói bởi người bán  
  - **tiki_delivery**: Hàng do TIKI sở hữu, chuyển từ (kho TIKI → khách hàng)  
- **brand**: Thương hiệu (OEM hoặc thương hiệu đã đăng ký).  
  - *OEM* nghĩa là nhà sản xuất thiết bị gốc, hoặc hiểu đơn giản là "không có thương hiệu"  
- **review_counts**: Số lượng đánh giá  
- **rating_average**: Điểm đánh giá trung bình  
- **favorite_count**: Số người dùng đã thêm sản phẩm này vào danh sách yêu thích  
- **pay_later**: Có hỗ trợ chương trình "mua trước trả sau" hay không  
- **current_seller**: Tên người bán hiện tại  
- **date_created**: Số ngày giữa ngày tạo và ngày cập nhật cuối cùng  
- **number_of_images**: Số lượng hình ảnh sản phẩm đã tải lên  
- **vnd_cashback**: Số tiền hoàn lại (cashback) tính theo VNĐ  
- **has_video**: `True` nếu có video, `False` nếu không  
- **quantity_sold**: Tổng số lượng đã bán trong lịch sử  


In [10]:
df["date_created_temp"] = df["loaded_date"] - pd.to_timedelta(df["date_created"], unit="D")

In [11]:
df = df.rename(columns={"date_created": "delta_days", "date_created_temp": "date_created"})

In [12]:
df[df.rating_average>0].rating_average.describe()

count    13017.000000
mean         4.413313
std          0.882090
min          1.000000
25%          4.100000
50%          4.700000
75%          5.000000
max          5.000000
Name: rating_average, dtype: float64

In [13]:
#df.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Products.csv", index=False, encoding="utf-8-sig")

## 1. Phân tích mô tả

### 1.1. Mô tả tình hình Trưng bày chết

In [13]:
t = df[df["id"] == "172192277"].values

In [14]:
df.columns

Index(['id', 'name', 'description', 'original_price', 'price',
       'fulfillment_type', 'brand', 'review_count', 'rating_average',
       'favourite_count', 'pay_later', 'current_seller', 'delta_days',
       'number_of_images', 'vnd_cashback', 'has_video', 'category',
       'quantity_sold', 'source', 'loaded_date', 'product_source',
       'date_created'],
      dtype='object')

In [15]:
df01 = df[["id", "name", "category", "product_source", "quantity_sold", "date_created"]]

In [16]:
df01["status"] = (df01.quantity_sold>0).astype(int)

In [17]:
df01[df01.quantity_sold > 0].shape

(20031, 7)

In [18]:
df01[df01.quantity_sold == 0].shape

(21550, 7)

In [19]:
df01.status.value_counts()

status
0    21550
1    20031
Name: count, dtype: int64

In [20]:
df01.head(2)

,id,name,category,product_source,quantity_sold,date_created,status
0,179731375,Balo nữ da dầy thời trang unisex màu sắc tươi ...,Balo nữ,Balo & vali,0,2023-03-18,0
1,249221065,Ví da đựng thẻ ngân hàng (7 màu) - 9205,Root,Balo & vali,0,2024-02-03,0


In [21]:
#df01.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Products_DisplayStatus.csv", index=False, encoding="utf-8-sig")

### 1.2. Báo cáo doanh thu

In [22]:
df02 = df[["id", "name", "product_source", "category", "original_price", "quantity_sold", "current_seller", "date_created"]]

In [23]:
df02["revenue"] = df02.original_price * df02.quantity_sold

In [24]:
df02.head()

,id,name,product_source,category,original_price,quantity_sold,current_seller,date_created,revenue
0,179731375,Balo nữ da dầy thời trang unisex màu sắc tươi ...,Balo & vali,Balo nữ,14250,0,Thiên Ân Balo,2023-03-18,0
1,249221065,Ví da đựng thẻ ngân hàng (7 màu) - 9205,Balo & vali,Root,10260,0,SHOP THỜI TRANG 2N,2024-02-03,0
2,102862196,Balo nam nữ thời trang họa tiết bò sữa kiểu m...,Balo & vali,Root,19999,0,Xưởng may balo Đà Nẵng,2022-04-09,0
3,115740385,Vải nhũ bóng simili chống thấm may túi ví balo...,Balo & vali,Root,30000,0,BB Outlet,2022-06-04,0
4,102862375,Combo balo vừa a4 và túi tote thời trang kiểu...,Balo & vali,Root,29000,0,Xưởng may balo Đà Nẵng,2022-04-09,0


In [25]:
df02.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Products_Revenue.csv", index=False, encoding="utf-8-sig")

### 1.3. Báo cáo đánh giá sản phẩm

In [26]:
df.columns

Index(['id', 'name', 'description', 'original_price', 'price',
       'fulfillment_type', 'brand', 'review_count', 'rating_average',
       'favourite_count', 'pay_later', 'current_seller', 'delta_days',
       'number_of_images', 'vnd_cashback', 'has_video', 'category',
       'quantity_sold', 'source', 'loaded_date', 'product_source',
       'date_created'],
      dtype='object')

In [27]:
df03 = df[["id", "name", "product_source", "category", "review_count", "rating_average", "current_seller"]]

In [28]:
df03["is_reviewed"] = (df03.rating_average>=1).astype(int)

In [29]:
#df03.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Products_Rating.csv", index=False, encoding="utf-8-sig")

### 1.4. Mô tả tình hình theo thời gian

In [30]:
df04 = df[["id", "name", "quantity_sold", "brand", "product_source", "category", "date_created", "current_seller"]]

In [31]:
#df04.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Products_Quantity.csv", index=False, encoding="utf-8-sig")

In [32]:
df04.head()

,id,name,quantity_sold,brand,product_source,category,date_created,current_seller
0,179731375,Balo nữ da dầy thời trang unisex màu sắc tươi ...,0,OEM,Balo & vali,Balo nữ,2023-03-18,Thiên Ân Balo
1,249221065,Ví da đựng thẻ ngân hàng (7 màu) - 9205,0,OEM,Balo & vali,Root,2024-02-03,SHOP THỜI TRANG 2N
2,102862196,Balo nam nữ thời trang họa tiết bò sữa kiểu m...,0,OEM,Balo & vali,Root,2022-04-09,Xưởng may balo Đà Nẵng
3,115740385,Vải nhũ bóng simili chống thấm may túi ví balo...,0,OEM,Balo & vali,Root,2022-06-04,BB Outlet
4,102862375,Combo balo vừa a4 và túi tote thời trang kiểu...,0,OEM,Balo & vali,Root,2022-04-09,Xưởng may balo Đà Nẵng


## 2. Phân tích chẩn đoán: Root Cause

### 2.1. Hiện tượng trưng bày chết

In [33]:
df.columns

Index(['id', 'name', 'description', 'original_price', 'price',
       'fulfillment_type', 'brand', 'review_count', 'rating_average',
       'favourite_count', 'pay_later', 'current_seller', 'delta_days',
       'number_of_images', 'vnd_cashback', 'has_video', 'category',
       'quantity_sold', 'source', 'loaded_date', 'product_source',
       'date_created'],
      dtype='object')

In [34]:
df05 = df[["id", "name", "description", "original_price", "brand", "product_source", "category", "quantity_sold", "date_created", "has_video", "number_of_images"]]

In [35]:
df05["status"] = (df05.quantity_sold > 0).astype(int)

In [36]:
df05["name_length"] = df05.name.str.len()

df05["has_video"] = df05["has_video"].astype(int)

In [38]:
#df05.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Status_RootCause.csv", index=False, encoding="utf-8-sig")

In [40]:
df05[["name_length", "number_of_images", "has_video", "status"]].head(3)

,name_length,number_of_images,has_video,status
0,55,10,0,0
1,39,2,0,0
2,98,5,0,0


### 2.2. Hiện tượng rating thấp (1-2)

In [49]:
df06 = df[["id", "name", "description", "rating_average", "original_price", "brand", "product_source", "category", "current_seller", "date_created", "has_video", "number_of_images"]]

In [50]:
df06["name_length"] = df05.name.str.len()

df06["has_video"] = df05["has_video"].astype(int)

In [51]:
df06.to_csv("D:/career/master/data_analysis/ck/Advanced_Data_Analyst/data/Rating_RootCause.csv", index=False, encoding="utf-8-sig")

## 3. Phân tích dự đoán: Recommendation System - Content-based filtering

In [15]:
df07 = df[["id", "name", "description", "original_price", "brand", "product_source", "category", "rating_average", "current_seller"]]

In [54]:
final_vector

array([0.2, 0.3, 0.4, 0.1, 4.2, 3. , 0. , 0. , 1. , 0. ])

In [17]:
df07.to_csv("Recommendation.csv", index=False, encoding="utf-8-sig")